<a href="https://colab.research.google.com/github/surajsy/ColabNotebooks/blob/master/EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.version

'3.6.9 (default, Apr 18 2020, 01:56:04) \n[GCC 8.4.0]'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

import scipy.io as sio
import glob
import numpy as np
import pandas as pd

In [ ]:
path = '/content/gdrive/My Drive/PRNN/EEG/eeg_data.csv'
data = pd.read_csv(path)
data = data.drop(['Unnamed: 0'], axis=1)

In [ ]:
X = data.iloc[:,:-1].to_numpy().astype(np.float32)
y = data.iloc[:,-1].to_numpy() - 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
np.shape(X)

(11500, 178)

In [ ]:
print(len(X.T))

178


In [ ]:
def train_fnn(train_loader, net, num_epochs, reshape, use_cuda=True):
  for epoch in range(num_epochs):
      for i, (data, labels) in enumerate(train_loader):
        data = Variable(data.view(reshape))         
        labels = Variable(labels)
          
        if use_cuda and torch.cuda.is_available():
            data = data.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             
        outputs = net(data)
        # print(np.shape(outputs), np.shape(labels))                            
        loss = criterion(outputs.squeeze(axis=0), labels)                 
        loss.backward()                                   
        optimizer.step()                                  
        
        # if (i+1) % 10 == 0:                             
        #     print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
        #         %(epoch+1, num_epochs, i+1, len(data)//batch_size, loss.item()))

def test_fnn(test_loader, net, reshape, use_cuda=True):              
  correct = 0
  total = 0
  for data, labels in test_loader:
    data = Variable(data.view(reshape))
    # print(np.shape(data))
    if use_cuda and torch.cuda.is_available():
        data = data.cuda()
        labels = labels.cuda()
    
    outputs = net(data)
    outputs = outputs.squeeze(axis=0)
    # print(np.shape(outputs))
    _, predicted = torch.max(outputs.data, 1)
    # print(np.shape(predicted))
    total += labels.size(0)                    
    correct += (predicted == labels).sum()     
      
  print('Accuracy of the network on the 3450 test samples: %d %%' % (100 * correct / total))

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size[0]) 
        self.relu = nn.ReLU()                          
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.elu = nn.ELU()
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[2])
        self.tanh = nn.Tanh()
        self.fc4 = nn.Linear(hidden_size[2],hidden_size[3])
        self.relu1 = nn.ReLU()
        self.fc5 = nn.Linear(hidden_size[3], num_classes)
    
    def forward(self, x):                              
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.elu(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.relu1(out)
        out = self.fc5(out)
        return out

In [ ]:
class ConvNet(nn.Module):
  def __init__(self, input_size, num_classes):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv1d(1, 32, 3)
    self.maxpool1 = nn.MaxPool1d(3, stride=2)
    self.tanh = nn.Tanh()
    self.conv2 = nn.Conv1d(32, 64, 3)
    self.maxpool2 = nn.MaxPool1d(3, stride=2)
    self.elu = nn.ELU()
    self.conv3 = nn.Conv1d(64, 64, 5)
    self.maxpool3 = nn.MaxPool1d(3, stride=2)
    self.relu = nn.ReLU()
    self.conv4 = nn.Conv1d(64, 64, 5)
    self.maxpool4 = nn.MaxPool1d(3, stride=2)
    self.elu1 = nn.ELU()
    self.fc1 = nn.Linear(64*6, 64)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(64, num_classes)

  def forward(self, x):
    out = self.conv1(x)
    out = self.maxpool1(out)
    out = self.tanh(out)

    out = self.conv2(out)
    out = self.maxpool2(out)
    out = self.elu(out)

    out = self.conv3(out)
    out = self.maxpool3(out)
    out = self.relu(out)

    out = self.conv4(out)
    out = self.maxpool4(out)
    out = self.elu1(out)
    
    out = out.view(-1, 64*6)

    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [ ]:
class LSTMnet(nn.Module):
  def __init__(self, hidden_size, num_layers, num_classes):
    super(LSTMnet, self).__init__()
    self.embedding = nn.Embedding(10000, 256, max_norm=2)
    self.lstm = nn.LSTM(256, hidden_size, num_layers, batch_first=True)
    self.dropout = nn.Dropout(p=0.1)
    self.fc1 = nn.Linear(hidden_size, 1024)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(1024, 256)
    self.tanh = nn.Tanh()
    self.fc3 = nn.Linear(256, num_classes)

  def forward(self, x):
    out = self.embedding(x)
    _, (out, cn) = self.lstm(out)
    out = self.dropout(out)
    out = self.fc1(out)
    out = self.tanh(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    return out

In [ ]:
train, test = torch.utils.data.random_split(list(zip(X, y)), [8050, 3450])

input_size    = 178
hidden_size   = [256, 256, 256, 256]
num_classes   = 5
num_epochs    = 25     
batch_size    = 250   
learning_rate = 1e-3

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

net = Net(input_size, hidden_size, num_classes)
use_cuda = True

if use_cuda and torch.cuda.is_available():
    net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

train_fnn(train_loader, net, num_epochs, [-1, input_size])
test_fnn(test_loader, net,  [-1, input_size])

Accuracy of the network on the 3450 test samples: 64 %


In [ ]:
test_fnn(train_loader)

TypeError: ignored

In [ ]:
train, test = torch.utils.data.random_split(list(zip(X, y)), [8050, 3450])

input_size    = 178
num_classes   = 5
num_epochs    = 100     
batch_size    = 250   
learning_rate = 5e-3

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

conv_net = ConvNet(input_size, num_classes)
use_cuda = True

if use_cuda and torch.cuda.is_available():
    conv_net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv_net.parameters(), lr=learning_rate)

train_fnn(train_loader, conv_net, num_epochs, [-1,1,input_size])
test_fnn(test_loader, conv_net, [-1,1,input_size])

Accuracy of the network on the 3450 test samples: 68 %


In [ ]:
X = data.iloc[:,:-1].to_numpy().astype(np.long)
y = data.iloc[:,-1].to_numpy() - 1

In [ ]:
X.max() - X.min()

3932

In [ ]:
train, test = torch.utils.data.random_split(list(zip(X, y)), [8050, 3450])

input_size    = 1
seq_len       = 178  
hidden_size   = 512
num_layers = 1
num_classes   = 2
num_epochs    = 25     
batch_size    = 250   
learning_rate = 1e-3

lstm_net = LSTMnet(hidden_size, num_layers, num_classes)

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

use_cuda = False

if use_cuda and torch.cuda.is_available():
    lstm_net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_net.parameters(), lr=learning_rate)

train_fnn(train_loader, lstm_net, num_epochs, [-1, seq_len], use_cuda)
test_fnn(test_loader, lstm_net, [-1, seq_len], use_cuda)

IndexError: ignored

In [ ]:
test_fnn(test_loader, lstm_net, [-1, seq_len, input_size])

torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([250, 178, 1])
torch.Size([250, 2])
torch.Size([250])
torch.Size([200, 178, 1])
torch.Size([200, 2])
torch.Size([200])
Accuracy of the network on the 3450 test samples: 30 %


In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
kernel = ['rbf']
C = [0.1, 1, 10]

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y,test_size=0.3,shuffle=True)

for c in C:
  for k in kernel:
    clf = SVC(C=c, kernel=k)

    clf.fit(train_X, train_y)
    pred = clf.predict(test_X)
    print('kernel:', k, '	C:', c, '	SVM Accuracy:', 100*accuracy_score(test_y,pred))

kernel: rbf 	C: 0.1 	SVM Accuracy: 44.028985507246375
kernel: rbf 	C: 1 	SVM Accuracy: 50.4927536231884
kernel: rbf 	C: 10 	SVM Accuracy: 58.666666666666664
